In [1]:
import cv2
import tensorflow as tf
import numpy as np

In [2]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [36]:
batch_size = 32
img_height = 32
img_width = 32

In [33]:
path="spectrograms"
train_ds = tf.keras.utils.image_dataset_from_directory(path,color_mode="grayscale",image_size=(32,32),validation_split=0.10,subset="training",seed=123,label_mode = "categorical")
val_ds = tf.keras.utils.image_dataset_from_directory(path,color_mode="grayscale",image_size=(32,32),validation_split=0.10,subset="validation",seed=123,label_mode = "categorical")

Found 8735 files belonging to 10 classes.
Using 7862 files for training.
Found 8735 files belonging to 10 classes.
Using 873 files for validation.


In [34]:
def normalization(image,label):
    image = tf.cast(image/255. , tf.float32)
    return image,label

In [35]:
normalized_train_ds = train_ds.map(normalization)
normalized_val_ds = val_ds.map(normalization)

In [7]:
normalized_train_ds = normalized_train_ds.unbatch()
images = list(normalized_train_ds.map(lambda x, y: x))
labels = list(normalized_train_ds.map(lambda x, y: y))

In [8]:
X_train,X_test,y_train,y_test = train_test_split(images,labels,test_size = 0.2)

In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

In [17]:
model=Sequential()
model.add(Conv2D(32,(3,3),input_shape=(32,32,32,),
    padding='same',activation='relu',
    kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Conv2D(32,(3,3),activation='relu',padding='same',kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512,activation='relu',kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [19]:
sgd=SGD(lr=0.01,momentum=0.9,decay=(0.01/25),nesterov=False)
 
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

In [20]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 dropout_2 (Dropout)         (None, 32, 32, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               4194816   
                                                      

In [28]:
for x, y in normalized_train_ds.take(1):
    print(x.shape, y)

(32, 32, 1) tf.Tensor([0. 0. 1. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float32)


In [31]:
class_names = train_ds.class_names
print(class_names)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [32]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(32, 32, 32, 1)
(32, 10)
